A demo of training, predicting and testing a linear classifier using the centered moments of simulated response data. Here we use a L1 regularizor to select the most expressive moments.


In [ ]:
# Dependencies
using StatsBase
using LaTeXStrings
using JLD
using GLMNet
using Plots 

In [ ]:
# include Machine learning definitions and functions
include("../src/MultiplyScattering_learn_moments.jl");

In [23]:
# Load training data
dataname = "bunnytime8";
datatestname = "bunnytimetest8";
train = load_moments(string(dataname,".mnts")); #bunnytime8  # bunnytime-low.
# Load testing data
test = load_moments(string(datatestname,".mnts")); #bunnytimetest8   # bunnytime-extra  # bunnytimetest-extra # bunnytimetest-low

loading:/home/rgower/Dropbox/Scattering-1.5D/numerics/mom_linear_regression/test/../../Data/bunnytime8.mnts.jld
loading:/home/rgower/Dropbox/Scattering-1.5D/numerics/mom_linear_regression/test/../../Data/bunnytimetest8.mnts.jld


In [58]:
## Using Lasso as a feature selection for the moments
## Learning the radius with Lasso and a feature map
momnums = [1,2,3,4,5,6];
optionsr = OptionsML(momnums,[],true) 
#Choose a feature map: fmmean, fmmoments,  fmofm
fmr = fmmoments; 
lambda = 0.0; # 0.0 for default choice of cross-validation grid, "GLMNet" lets GLMNet choose the grid
mlr = setup_ml(train, fmr, "L1",optionsr, "radius");
# fitting the radius model
fit_L1(mlr, optionsr, lambda);
Rsqdr = test_linear(test, datatestname, fmr,  mlr,optionsr, "radius");

(#data, #features): (205, 59)
Fitting momentsmapL1-radius
23.6% absolute train error
radius 17.6% absolute test error
R squared error radius= 0.82


In [51]:
## Plotting the weight attributed to eat moment. The plot shows that the 3rd moment is not important
lengthresp = length(train[1].moments[1]);
smon = size(train[1].moments[momnums]);
mom_weights_square_r = reshape( abs.(mlr.parameters[1:end-1]), smon[1], lengthresp);
heatmap(train[1].x_arr,1:smon[1], mom_weights_square_r', xlabel="time", ylabel="moment",fill=(true,cgrad(:blues,[0,0.1,1.0])))#
savefig("../figures/weights-heatmap-$(mlr.name).pdf");

In [54]:
## Using Lasso as a feature selection for the moments
## Learning the concentration with Lasso and a feature map
momnums = [1,2,3,4,5,6]; 
optionsv = OptionsML(momnums,[],true) 
#Choose a feature map: fmmean, fmmoments,  fmofm,
fmv = fmmoments; 
lambda = 0.0; # 0.0 for default choice of cross-validation grid, "GLMNet" lets GLMNet choose the grid
# retrieving lambdas that were calculated using  cross validation. See cross_valid.ipynb
mlv = setup_ml(train, fmv, "L1",optionsv, "concen");
# fitting the concentration model
fit_L1(mlv, optionsv, lambda);
Rsqdr = test_linear(test, datatestname, fmv,  mlv,optionsv, "concen");

(#data, #features): (205, 175)
Fitting momentsmapL1-concen
8.3% absolute train error
concen 22.0% absolute test error
R squared error concen= 0.93


In [55]:
## Plotting the weight attributed to eat moment. The plot shows that the 3rd moment is not important
lengthresp = length(train[1].moments[1]);
smon = size(train[1].moments[momnums]);
mom_weights_square_v = reshape( abs.(mlv.parameters[1:end-1]), smon[1], lengthresp);
heatmap(train[1].x_arr,1:smon[1], mom_weights_square_v', xlabel="time", ylabel="moment",fill=(true,cgrad(:blues,[0,0.1,1.0])))#
savefig("../figures/weights-heatmap-$(mlv.name).pdf");

In [56]:
## Predict a single particle using the kernal ridgeregression radius and volume models
particlenum = 10;
vl = test[particlenum].label[1]
rd = test[particlenum].label[2]
println("(vl, rd) =",vl,',',rd);
# Make a single prediction on test set
 vpred = predict_linear(test[particlenum], fmv,  mlv, optionsv)
 rpred = predict_linear(test[particlenum], fmr,  mlr, optionsr)
 println("(vpred, rpred) =",floor(vpred[1]*1000)/1000,',',floor(rpred[1]*1000)/1000);

(vl, rd) =0.03,1.11
(vpred, rpred) =0.023,1.062


In [57]:
## Using Lasso as a feature selection for the moments
## Learning the volume fraction with Lasso and a feature map
momnums = [1,2,3,4,5,6];
optionsv = OptionsML(momnums,[],true) 
#Choose a feature map: fmmean, fmmoments,  fmofm,
fmv = fmmoments; 
lambda = 0.0; # 0.0 for default choice of cross-validation grid, "GLMNet" lets GLMNet choose the grid
# retrieving lambdas that were calculated using  cross validation. See cross_valid.ipynb
mlv = setup_ml(train, fmv, "L1",optionsv, "volumefraction");
# fitting the concentration model
fit_L1(mlv, optionsv, lambda);
Rsqdr = test_linear(test, datatestname, fmv,  mlv,optionsv, "volumnefraction");

(#data, #features): (205, 175)
Fitting momentsmapL1-volumefraction
32.1% absolute train error
volumnefraction 83.9% absolute test error
R squared error volumnefraction= 0.01
